In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
#Create and mount on virtual gdrive through Colab

#데이터 불러오기

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import io
import glob
from PIL import Image
import dlib
import cv2
from google.colab.patches import cv2_imshow
from skimage.color import rgb2gray

#Dataset

### Dataset load

Downloaded files from github. Each of the zip files extracted on gdrive

In [ ]:
path_to_zip_file = '/gdrive/My Drive/BNCS411/mask_detection_report/with_mask.zip'
directory_to_extract_to = '/gdrive/My Drive/BNCS411/mask_detection_report/ds/with_mask2'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = '/gdrive/My Drive/BNCS411/mask_detection_report/without_mask.zip'
directory_to_extract_to = '/gdrive/My Drive/BNCS411/mask_detection_report/ds/without_mask2'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

# Preprocessing

###Localize face boundary

Import pretrained ssd model

Define it as facenet, find each of the dataset's facial area

In [ ]:
from tensorflow.keras.models import load_model
facenet = cv2.dnn.readNet('/gdrive/MyDrive/BNCS411/mask_detection_report/model/deploy.prototxt', 
                          '/gdrive/MyDrive/BNCS411/mask_detection_report/model/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
mask_faces = []
for i in glob.glob('/gdrive/MyDrive/BNCS411/mask_detection_report/with_mask2/*.jpg'):
    test = cv2.imread(i)
    h, w = test.shape[:2]

    #load_model('/gdrive/MyDrive/BNCS411/mask_detection_report/mask_detector.model')
    blob = cv2.dnn.blobFromImage(test, 1.0, size=(150,150), mean=(104.0, 177.0, 123.0), crop = False)
    facenet.setInput(blob)
    dets = facenet.forward()

    max = 0
    argmax = 0
    for j in range(dets.shape[2]):
        confidence = dets[0, 0, j, 2]
        if max < confidence:
            max = confidence
            argmax=j
    
    x1 = int(dets[0, 0, argmax, 3] * w)
    y1 = int(dets[0, 0, argmax, 4] * h)
    x2 = int(dets[0, 0, argmax, 5] * w)
    y2 = int(dets[0, 0, argmax, 6] * h)
    
    face = test[y1:y2, x1:x2]
    
    if np.sum(face)==0:
        continue
    face = cv2.resize(face, (224,224))
    mask_faces.append(face)



In [ ]:
unmask_faces = []
for i in glob.glob('/gdrive/MyDrive/BNCS411/mask_detection_report/without_mask2/*.jpg'):
    test = cv2.imread(i)
    h, w = test.shape[:2]

    #load_model('/gdrive/MyDrive/BNCS411/mask_detection_report/mask_detector.model')
    blob = cv2.dnn.blobFromImage(test, 1.0, size=(150,150), mean=(104.0, 177.0, 123.0), crop = False)
    facenet.setInput(blob)
    dets = facenet.forward()

    max = 0
    argmax = 0
    for j in range(dets.shape[2]):
        confidence = dets[0, 0, j, 2]
        if max < confidence:
            max = confidence
            argmax=j
    
    
    x1 = int(dets[0, 0, argmax, 3] * w)
    y1 = int(dets[0, 0, argmax, 4] * h)
    x2 = int(dets[0, 0, argmax, 5] * w)
    y2 = int(dets[0, 0, argmax, 6] * h)
    
    face = test[y1:y2, x1:x2]
    
    if np.sum(face)==0:
        continue
    face = cv2.resize(face, (224,224))
    unmask_faces.append(face)



In [ ]:
len(mask_faces)
len(unmask_faces)

In [ ]:
plt.figure(figsize=(22, 22))
for i, face in enumerate(unmask_faces):
        plt.subplot(27, 27, i+1)
        plt.imshow(face[:, :, ::-1])

In [ ]:
plt.figure(figsize=(22, 22))
for i, face in enumerate(mask_faces):
        plt.subplot(27, 27, i+1)
        plt.imshow(face[:, :, ::-1])

Save detected image files

In [ ]:
np.save('/gdrive/MyDrive/BNCS411/mask_detection_report/npds/unmask2.npy', unmask_faces)
np.save('/gdrive/MyDrive/BNCS411/mask_detection_report/npds/mask2.npy', mask_faces)

### Data augmentation

Load localized images

In [ ]:
unmask = np.load('/gdrive/MyDrive/BNCS411/mask_detection_report/npds/unmask2.npy')
mask = np.load('/gdrive/MyDrive/BNCS411/mask_detection_report/npds/mask2.npy')

In [ ]:
m_lb = mask.shape[0]
u_lb = unmask.shape[0]
masklb = [0]*m_lb
unmasklb = [1]*u_lb

In [ ]:
ds = np.concatenate((mask, unmask), axis = 0)
lb = np.concatenate((masklb, unmasklb))
ds.shape, lb.shape

Important thing is before generating augmented dataset, split data into train and validation set

In [ ]:
Xtrain, Xtest, trainlb, Ytest = train_test_split(ds, lb, test_size=0.2, random_state=42)

In [ ]:
image_gen = ImageDataGenerator(rotation_range = 30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale = 1/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

Check the generation working well

In [ ]:
fig, axs = plt.subplots(5,5,figsize = (10,10))
k=0
for i in range(5):
    for j in range(5):
        axs[i][j].imshow(image_gen.random_transform(Xtrain[k]))
        k+=1

Save as augmented data as 'aug_ds'

In [ ]:
aug_ds = []
for i in range(Xtrain.shape[0]):
    tmp = image_gen.random_transform(Xtrain[i])
    aug_ds.append(tmp)

aug_ds = np.array(aug_ds)

In [ ]:
aug_ds.shape

In [ ]:
fullds = np.concatenate((Xtrain, aug_ds), axis = 0)
Ytrain = np.concatenate((trainlb, trainlb))
fullds.shape, Ytrain.shape

### Normalizing

In [ ]:
ds_gray = rgb2gray(fullds) #grayscale
Xtest = rgb2gray(Xtest)
ds_gray.shape, Xtest.shape

In [ ]:
blur_fullds = [] #denoise and smoothing
blur_test = []
for i in range(ds_gray.shape[0]):
    blur = cv2.GaussianBlur(ds_gray[i, :,:], (5, 5), 0)
    blur_fullds.append(blur)
    
for i in range(Xtest.shape[0]):
    testblur = cv2.GaussianBlur(Xtest[i, :,:], (5, 5), 0)
    blur_test.append(testblur)

blur_fullds = np.array(blur_fullds, dtype=np.float32)
blur_test = np.array(blur_test, dtype=np.float32)

In [ ]:
#minmax scaling
stdds = []
Xtest = []
from sklearn.preprocessing import MinMaxScaler
for i in range(blur_fullds.shape[0]):
    scaler = MinMaxScaler()
    stdds.append(scaler.fit_transform(blur_fullds[i,:,:]))

for i in range(blur_test.shape[0]):
    scaler = MinMaxScaler()
    Xtest.append(scaler.fit_transform(blur_test[i,:,:]))

stdds = np.array(stdds)
stdts = np.array(Xtest)
stdds.shape, stdts.shape

In [ ]:
fig, axs = plt.subplots(5,5,figsize = (10,10))
k=0
for i in range(5):
    for j in range(5):
        axs[i][j].imshow(stdds[k])
        k+=1

In [ ]:
Xtrain = np.expand_dims(stdds, axis=-1)
Xtest = np.expand_dims(stdts, axis=-1)
Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

# CNN model

In [ ]:
model = Sequential([
    keras.layers.Conv2D(input_shape=(224,224,1), filters=15, kernel_size=3, strides=1, padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(filters=20, kernel_size=3, strides=1, padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(60, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer=Adam(learning_rate=0.00001), loss = 'binary_crossentropy', metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "/gdrive/MyDrive/BNCS411/mask_detection_report/my_modelaug.h5", save_best_only=True
)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5, min_lr=0)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=10)

In [ ]:
model.summary()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
%matplotlib inline

SVG(model_to_dot(model, show_shapes=True, dpi=60).create(prog='dot', format='svg', ))

In [ ]:
history = model.fit(Xtrain, Ytrain, validation_data=(Xtest, Ytest), epochs=10, callbacks = [early_stopping_cb, learning_rate_reduction, checkpoint_cb])

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(2,1,1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(2,1,2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#Test

###Test for images

In [ ]:
weight_path = "/gdrive/MyDrive/BNCS411/mask_detection_report/my_modelaug.h5"

model = tf.keras.models.load_model(weight_path)

In [ ]:
test = []
for i in glob.glob('/gdrive/MyDrive/BNCS411/mask_detection_report/test/*.jpg'):
    img = cv2.imread(i)
    h, w = img.shape[:2]

    #load_model('/gdrive/MyDrive/BNCS411/mask_detection_report/mask_detector.model')
    blob = cv2.dnn.blobFromImage(img, 1.0, size=(150,150), mean=(104.0, 177.0, 123.0), crop = False)
    facenet.setInput(blob)
    dets = facenet.forward()

    max = 0
    argmax = 0
    for j in range(dets.shape[2]):
        confidence = dets[0, 0, j, 2]
        if max < confidence:
            max = confidence
            argmax=j
    
    x1 = int(dets[0, 0, argmax, 3] * w)
    y1 = int(dets[0, 0, argmax, 4] * h)
    x2 = int(dets[0, 0, argmax, 5] * w)
    y2 = int(dets[0, 0, argmax, 6] * h)
    
    face = img[y1:y2, x1:x2]
    
    if np.sum(face)==0:
        continue
    face = cv2.resize(face, (224,224))
    test.append(face)


In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(test[6])

In [ ]:
test = np.array(test)
test = rgb2gray(test)

In [ ]:
blur_testimg = []
for i in range(test.shape[0]):
    blur = cv2.GaussianBlur(test[i, :,:], (5, 5), 0)
    blur_testimg.append(blur)

blur_testimg = np.array(blur_testimg, dtype=np.float32)

In [ ]:
full = []
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
for i in range(blur_testimg.shape[0]):
    scaler = StandardScaler()
    full.append(scaler.fit_transform(blur_testimg[i,:,:]))

fullts = np.array(test)
fullts.shape

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(7):
    plt.subplot(3, 3, i+1)
    plt.imshow(test[i, :, :])

In [ ]:
fullts = np.expand_dims(fullts, axis=-1)
fullts.shape

In [ ]:
model.predict(fullts)

###Test with webcam


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
vid = open('/content/test.mp4')
vid_in = cv2.VideoCapture('/content/test.mp4')

while True:
    ret, image = vid_in.read()
    
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, size=(300,300), mean=(104.0, 177.0, 123.0), crop = False)
    facenet.setInput(blob)
    dets = facenet.forward()

    faces = []

    for j in range(dets.shape[2]):
        confidence = dets[0, 0, j, 2]
        if confidence < 0.5:
            continue


        x1 = int(dets[0, 0, j, 3] * w)
        y1 = int(dets[0, 0, j, 4] * h)
        x2 = int(dets[0, 0, j, 5] * w)
        y2 = int(dets[0, 0, j, 6] * h)

        face = image[y1:y2, x1:x2]
        faces.append(face)

        for i, face in enumerate(faces):
            face_input = cv2.resize(face, (224,224))
            face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2GRAY)
            face_input = cv2.GaussianBlur(face_input, (5,5), 0)
            face_input = scaler.fit_transform(face_input)
            face_input = np.expand_dims(face_input, axis=0)
            face_input = np.expand_dims(face_input, axis=3)

            pred = model.predict_classes(face_input)

            if pred[0][0] == 0:
                cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 3)
                cv2.putText(image, 'Mask On', (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

            else:
                cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 3)
                cv2.putText(image, 'Mask Off', (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

    
    cv2_imshow(image)
    
    key = cv2.waitKey(1)
    
    if key == 27:
        break

vid_in.release()
cv2.destroyAllWindows()